classifiers: nn.crossentropyloss = -log-likelihood --- use for logp(y) and -logq(y/x) for support <br>
kl-div: <br>
reconstr-loss: set reduction to none and then take mean of losses per image in the total batch. This gives reconstr-loss per image for further computation<br> 

In [1]:
import sys
import random
import numpy as np
import torch
import learn2learn as l2l
import pandas as pd

from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from PIL.Image import LANCZOS

In [2]:
torch.cuda.is_available()

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [2]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.9.1101343141/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.9.1101343141/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [3]:
#from data.loaders import Omniglotmix, MiniImageNet
from data.taskers import gen_tasks
from src.zoo.archs import CCVAE, CEncoder, Classifier_VAE, ConvBase
from src.zoo.delpo_utils import setup, inner_adapt_delpo, loss, accuracy
from src.utils2 import Profiler

In [5]:
train_tasks = gen_tasks(dataname='miniimagenet', root='../../dataset/mini_imagenet', mode='train', n_ways=5, k_shots=5, q_shots=15, image_transforms=None)

In [5]:
classes = list(range(1623))
random.shuffle(classes)
image_transforms = transforms.Compose([transforms.Resize(28, interpolation=LANCZOS),
                                                    transforms.ToTensor(),
                                                    lambda x: 1.0 - x,
                                                ])
train_tasks = gen_tasks('omniglot', '/home/anuj/Desktop/Work/TU_Delft/research/implement/dataset/omniglot', image_transforms=image_transforms, n_ways=5, k_shots=5, q_shots=15, classes=classes[:1100])

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [10]:
task = train_tasks.sample()

NameError: name 'train_tasks' is not defined

In [13]:
task[0].mean()

tensor(0.0879)

In [10]:
data, labels = task
data, labels = data.to('cpu') / 255.0, labels.to('cpu')
total = 5 * (5 + 15)
queries_index = np.zeros(total)

# Extracting the evaluation datums from the entire task set, for the meta gradient calculation
for offset in range(5):
    queries_index[np.random.choice(
        5+15, 15, replace=False) + ((5 + 15)*offset)] = True
support = data[np.where(queries_index == 0)]
support_labels = labels[np.where(queries_index == 0)]
queries = data[np.where(queries_index == 1)]
queries_labels = labels[np.where(queries_index == 1)]

In [11]:
queries_labels

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1])

In [12]:
abc = torch.load('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp3/latents_epoch-0_train.pt')
abc

{'label_latents': [tensor([[ 0.6558, -0.1321, -0.2266,  ..., -0.8036,  0.5112,  0.1200],
          [ 0.7546,  0.1008, -0.1327,  ..., -0.4581, -0.6979, -0.8913],
          [ 1.1275,  0.2353, -0.3868,  ..., -0.2003, -0.2956, -0.6518],
          ...,
          [ 0.3954,  0.0117,  0.2331,  ..., -0.5195,  1.5961,  0.0703],
          [ 0.5831,  0.1810, -0.2826,  ...,  0.0421,  0.2497, -0.3877],
          [ 0.0236,  0.0432, -0.5602,  ..., -0.3434,  0.5206, -0.4129]]),
  tensor([[ 0.3233,  0.0171, -0.9282,  ...,  0.2394, -0.7846,  0.1493],
          [ 0.4558,  0.5093, -0.5382,  ..., -0.3772, -0.6001, -0.2965],
          [-0.3069,  0.5045, -0.4573,  ...,  0.2668, -0.0322,  0.0196],
          ...,
          [ 0.1328,  0.2652, -0.8606,  ..., -0.5518, -0.6888, -0.0732],
          [-0.1698, -0.1367, -0.5738,  ..., -0.0027, -0.1081,  0.2523],
          [ 0.1448,  0.5585, -0.6425,  ...,  0.5092, -0.6544,  0.4169]])],
 'style_latents': [tensor([[-0.0756,  0.1746,  0.2820,  ..., -0.3079, -0.5900,  0.22

In [4]:
device = 'cpu'
n_ways = 5
k_shots = 5
q_shots = 15
dataset = 'omniglot'
root = '../../dataset/omniglot'
order = False
inner_lr = 0.01
meta_lr = 0.003
reconst_loss = nn.MSELoss(reduction='none')
inner_adapt_steps_train = 5
meta_batch_size = 10

In [5]:
train_tasks, valid_tasks, test_tasks, learner = setup(
    dataset, root, n_ways, k_shots, q_shots, order, inner_lr, device, download='False', repar=True)
opt = optim.Adam(learner.parameters(), meta_lr)

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [11]:
a = learner(train_tasks.sample()[0])
a[2].shape

torch.Size([100, 64])

In [16]:
ttask = train_tasks.sample()
ttask

[tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         ...,
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ..

In [19]:
model = ConvBase(1, 64, 4 ,False, (2,2))
a = model(ttask[0])
a.shape

torch.Size([100, 64, 2, 2])

In [24]:
ttask[0].shape

torch.Size([100, 1, 28, 28])

In [ ]:
28 -> 14 -> 7 -> 4 -> 2

In [27]:
a2 = nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=2)(a2)
a2.shape

torch.Size([100, 64, 2, 2])

In [22]:
a1 = nn.Flatten()(a)
a1.shape

torch.Size([100, 256])

In [7]:
batch_losses = []

In [8]:
class Args:
    def __init__(self, wt_ce, klwt, rec_wt, beta_l, beta_s):
        #args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s
        self.wt_ce = wt_ce
        self.klwt = klwt
        self.rec_wt = rec_wt
        self.beta_l = beta_l
        self.beta_s = beta_s
        self.dataset = 'omniglot'

In [9]:
args = Args(100, False, 0.1, 1, 1)

In [40]:
ttask = train_tasks.sample()
model = learner.clone()
evaluation_loss, evaluation_accuracy, reconst_img, query_imgs, mu_l, log_var_l, mu_s, log_var_s = inner_adapt_delpo(
                ttask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device, True, args)

In [41]:
evaluation_loss

{'elbo': tensor(171.9512, device='cuda:0', grad_fn=<AddBackward0>),
 'label_kl': tensor(0.1119, device='cuda:0', grad_fn=<MeanBackward0>),
 'style_kl': tensor(0.0178, device='cuda:0', grad_fn=<MeanBackward0>),
 'reconstruction_loss': tensor(63.6320, device='cuda:0', grad_fn=<MeanBackward0>),
 'classification_loss': tensor(1.6546, device='cuda:0', grad_fn=<NllLossBackward>)}

In [42]:
evaluation_loss['elbo'].backward()

In [33]:
tmp = [1, evaluation_accuracy.item()]
tmp = tmp + [a.item() for a in evaluation_loss.values()]
batch_losses.append(tmp)

In [34]:
batch_losses

[[1,
  0.20000000298023224,
  213.8948516845703,
  15.829955101013184,
  14.429597854614258,
  2242.850830078125,
  1.6120678186416626]]

In [10]:
learner = torch.load('./model.pt')
learner.to(device)
opt.load_state_dict(torch.load('./opt.pt'))

In [12]:
## Training ##
for iter in range(2):
    opt.zero_grad()
    batch_losses = []

    for batch in range(meta_batch_size):
        ttask = train_tasks.sample()
        model = learner.clone()
        if (batch == 0):
            evaluation_loss, evaluation_accuracy, reconst_img, query_imgs, mu_l, log_var_l, mu_s, log_var_s = inner_adapt_delpo(
                ttask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device, True, args)
            
            # Logging train-task images and latents
            # di = {"reconst_examples": reconst_img.detach().to('cpu'), "gt_examples": query_imgs.detach().to('cpu')}
            # dl = {"label_latents": [mu_l.detach().to('cpu'), log_var_l.detach().to('cpu')],
            #       "style_latents": [mu_s.detach().to('cpu'), log_var_s.detach().to('cpu')]}
            # profiler.log_data(di, iter, 'images', 'train')
            # profiler.log_data(dl, iter, 'latents', 'train')

        else:
            evaluation_loss, evaluation_accuracy = inner_adapt_delpo(
                ttask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device, False, args)

        evaluation_loss['elbo'].backward()

        # Logging per train-task losses and accuracies
        # tmp = [(iter*meta_batch_size)+batch, evaluation_accuracy.item()]
        # tmp = tmp + [a.item() for a in evaluation_loss.values()]
        # batch_losses.append(tmp)

    #     wandb.log(dict({f"train/{key}": loss.item() for _, (key, loss) in enumerate(evaluation_loss.items())},
    #               **{'train/accuracies': evaluation_accuracy.item(), 'train/task': (iter*args.meta_batch_size)+batch}))

    # rimages = wandb.Image(reconst_img, caption="Reconstructed Query Images")
    # qimages = wandb.Image(query_imgs, caption="Query Images")
    # wandb.log({"reconst_examples": rimages, "gt_examples": qimages})

    vtask = valid_tasks.sample()
    model = learner.clone()
    if iter % 2 == 0:
        validation_loss, validation_accuracy, reconst_img, query_imgs, mu_l, log_var_l, mu_s, log_var_s = inner_adapt_delpo(
            vtask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device, True, args)
        # # Logging valid-task images and latents
        # di = {"reconst_examples": reconst_img.to('cpu'), "gt_examples": query_imgs.to('cpu')}
        # dl = {"label_latents": [mu_l.to('cpu'), log_var_l.to('cpu')],
        #     "style_latents": [mu_s.to('cpu'), log_var_s.to('cpu')]}
        # profiler.log_data(di, iter, 'images', 'valid')
        # profiler.log_data(dl, iter, 'latents', 'valid')

    else:
        validation_loss, validation_accuracy = inner_adapt_delpo(
            vtask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device, False, args)

    # Logging per validation-task losses and accuracies
    # tmp = [iter, validation_accuracy.item()]
    # tmp = tmp + [a.item() for a in validation_loss.values()]

    # wandb.log(dict({f"valid/{key}": loss.item() for _, (key, loss) in enumerate(validation_loss.items())},
    #           **{'valid/accuracies': validation_accuracy.item(), 'valid/task': iter}))
# Meta backpropagation of gradients
    for p in learner.parameters():
        p.grad.data.mul_(1.0 / 5)
    opt.step()

    # Saving the Logs
    # profiler.log_csv(batch_losses, 'train')
    # profiler.log_csv(tmp, 'valid')

#torch.save(learner, f='../repro')


In [11]:
torch.save(learner.to('cpu'), './model.pt')
torch.save(opt.state_dict(), './opt.pt')    

In [13]:
opt.state_dict()

{'state': {0: {'step': 2,
   'exp_avg': tensor([[[[-2.9039e-05, -2.9700e-05, -2.6917e-05],
             [-3.0638e-05, -2.8049e-05, -2.5201e-05],
             [-1.2374e-05, -7.1534e-06, -7.9703e-06]]],
   
   
           [[[-1.1044e-06, -2.4809e-06, -2.5188e-06],
             [-1.0337e-05, -6.8135e-06, -5.1130e-06],
             [-1.7752e-05, -1.5771e-05, -1.5036e-05]]],
   
   
           [[[-5.1022e-05, -4.5249e-05, -3.6734e-05],
             [-6.2129e-05, -5.9803e-05, -4.8917e-05],
             [-6.9662e-05, -7.0925e-05, -6.2238e-05]]],
   
   
           [[[-1.2599e-05, -7.2536e-06, -4.3617e-06],
             [-1.3515e-05, -1.1269e-05, -1.0050e-05],
             [-9.5757e-06, -8.9942e-06, -1.1055e-05]]],
   
   
           [[[-2.5058e-07, -1.3343e-06, -1.0012e-06],
             [ 5.9340e-07,  9.5581e-08, -9.6288e-07],
             [ 1.2456e-06,  3.3525e-07, -6.5785e-07]]],
   
   
           [[[-3.2150e-05, -3.7914e-05, -4.0971e-05],
             [-3.0211e-05, -3.8502e-05, -4.1921e-

In [12]:
opt.state_dict()

{'state': {0: {'step': 2,
   'exp_avg': tensor([[[[-2.9039e-05, -2.9700e-05, -2.6917e-05],
             [-3.0638e-05, -2.8049e-05, -2.5201e-05],
             [-1.2374e-05, -7.1534e-06, -7.9703e-06]]],
   
   
           [[[-1.1044e-06, -2.4809e-06, -2.5188e-06],
             [-1.0337e-05, -6.8135e-06, -5.1130e-06],
             [-1.7752e-05, -1.5771e-05, -1.5036e-05]]],
   
   
           [[[-5.1022e-05, -4.5249e-05, -3.6734e-05],
             [-6.2129e-05, -5.9803e-05, -4.8917e-05],
             [-6.9662e-05, -7.0925e-05, -6.2238e-05]]],
   
   
           [[[-1.2599e-05, -7.2536e-06, -4.3617e-06],
             [-1.3515e-05, -1.1269e-05, -1.0050e-05],
             [-9.5757e-06, -8.9942e-06, -1.1055e-05]]],
   
   
           [[[-2.5058e-07, -1.3343e-06, -1.0012e-06],
             [ 5.9340e-07,  9.5581e-08, -9.6288e-07],
             [ 1.2456e-06,  3.3525e-07, -6.5785e-07]]],
   
   
           [[[-3.2150e-05, -3.7914e-05, -4.0971e-05],
             [-3.0211e-05, -3.8502e-05, -4.1921e-

In [20]:
log_var_l.shape

torch.Size([2, 64])

In [20]:
test_tasks.sample

[tensor([[[[179., 164., 166.,  ..., 219., 225., 229.],
           [162., 171., 162.,  ..., 212., 220., 218.],
           [154., 151., 161.,  ..., 216., 215., 215.],
           ...,
           [255., 255., 255.,  ..., 255., 255., 255.],
           [255., 255., 255.,  ..., 255., 255., 255.],
           [255., 255., 255.,  ..., 255., 255., 255.]],
 
          [[171., 160., 163.,  ..., 223., 232., 233.],
           [156., 166., 158.,  ..., 212., 230., 229.],
           [149., 144., 154.,  ..., 216., 227., 227.],
           ...,
           [255., 255., 255.,  ..., 255., 255., 255.],
           [255., 255., 255.,  ..., 255., 255., 255.],
           [255., 255., 255.,  ..., 255., 255., 255.]],
 
          [[157., 142., 155.,  ..., 235., 239., 243.],
           [137., 147., 146.,  ..., 220., 235., 238.],
           [128., 126., 135.,  ..., 225., 233., 238.],
           ...,
           [255., 255., 255.,  ..., 255., 255., 255.],
           [255., 255., 255.,  ..., 255., 255., 255.],
           

In [20]:
torch.save(learner, './model.pt')

In [22]:
learner = torch.load('./model.pt')

In [25]:
learner = learner.to('cpu')

In [27]:
learner(vtask[0])

(tensor([[[[0.5460, 0.7005, 0.7540,  ..., 0.6607, 0.5247, 0.6521],
           [0.3869, 0.4778, 0.6000,  ..., 0.5082, 0.4487, 0.5776],
           [0.5202, 0.7316, 0.7623,  ..., 0.5487, 0.4707, 0.5494],
           ...,
           [0.4386, 0.4401, 0.5506,  ..., 0.5411, 0.4927, 0.5433],
           [0.4770, 0.3676, 0.4371,  ..., 0.3584, 0.3767, 0.4090],
           [0.4787, 0.3858, 0.3679,  ..., 0.3389, 0.3394, 0.4149]],
 
          [[0.4559, 0.4518, 0.4888,  ..., 0.7013, 0.6391, 0.3140],
           [0.3996, 0.3678, 0.3866,  ..., 0.4096, 0.3197, 0.2149],
           [0.4501, 0.2640, 0.2461,  ..., 0.2523, 0.1991, 0.1818],
           ...,
           [0.2929, 0.2676, 0.3173,  ..., 0.3396, 0.2282, 0.2554],
           [0.3211, 0.2309, 0.2781,  ..., 0.4156, 0.2826, 0.2474],
           [0.5293, 0.4962, 0.4777,  ..., 0.4851, 0.3834, 0.4208]],
 
          [[0.7325, 0.5827, 0.5901,  ..., 0.7413, 0.6871, 0.7642],
           [0.7580, 0.6519, 0.6883,  ..., 0.7370, 0.6829, 0.7204],
           [0.7148, 0.49

In [11]:

## Testing ##
print('Testing on held out classes')

for i, tetask in enumerate(test_tasks):
    # wandb.define_metric("accuracies", summary="max")
    # wandb.define_metric("accuracies", summary="mean")

    model = learner.clone()
    #tetask = test_tasks.sample()
    evaluation_loss, evaluation_accuracy = inner_adapt_delpo(
        tetask, reconst_loss, model, n_ways, k_shots, q_shots, 5, device, False, args)

    # Logging per test-task losses and accuracies
    # tmp = [iter, evaluation_accuracy.item()]
    # tmp.append(a.item() for a in evaluation_loss.values())
    # profiler.log_csv(tmp, 'test')
    # # wandb.log(dict({f"test/{key}": loss.item() for _, (key, loss) in enumerate(evaluation_loss.items())},
    #             **{'test/accuracies': evaluation_accuracy.item(), 'test/task': i}))


Testing on held out classes


KeyboardInterrupt: 